## ICSI corpus. 
Go through the list of files and extract passage and the summaries, using the DialogueActs as intermediary

In [75]:
import glob, os
#import xml.etree.ElementTree as et
from lxml import etree as et
from collections import OrderedDict, defaultdict

### Extract words

In [46]:
meeting_dict = dict() # key = meeting, value = ordered dict of word id: word
cur_word = None
for word_f in sorted(glob.glob("/home/rajivn/W266/W266-fall-2020-hwu-rnair/data/ICSI_plus_NXT/ICSIplus/Words/*xml")):
    meeting_name = word_f.split("/")[-1].split(".")[0]
    root = et.parse(word_f).getroot()
    nodes = root.findall('*')
    meet_words = OrderedDict()

    for node in nodes:
        if 'W' in node.values():
            meet_words[node.attrib['{http://nite.sourceforge.net/}id']] = node.text
            cur_word = node.text
        for w in node.values():
            if "disfmarker" in w:
                meet_words [node.attrib['{http://nite.sourceforge.net/}id']] = cur_word
    meeting_dict[meeting_name] = meet_words
    


##### Extract complete text and store in files under ../ICSI_plus_NXT/processing/_meeting_id_words.txt file

In [94]:
len(meeting_dict.keys())

for meeting, words in meeting_dict.items():
    with open("../data/ICSI_plus_NXT/processing/"+meeting+"_words.txt", "w") as f:
        for word in words.values():
            f.write(word)
            f.write(" ")

### Extract dialogues

Dialogues are equivalent to complete sentences

In [96]:
meeting_dialogues = defaultdict(list) # key = meeting name, value = list of tuples start, end word ids(end can be same as start)
for dialog_f in sorted(glob.glob("/home/rajivn/W266/W266-fall-2020-hwu-rnair/data/ICSI_plus_NXT/ICSIplus/DialogueActs/*acts.xml")):
    meeting_name = dialog_f.split("/")[-1].split(".")[0]
    root = et.parse(dialog_f).getroot()
    dlist = root.findall('dialogueact/{http://nite.sourceforge.net/}child')
    for dl in dlist:
        words = dl.attrib['href']
        words = words.split("#")[1]
        try:
            start, end = words.split("..")
            start, end = start.replace("id", ""), end.replace("id", "")
            start, end = start.replace("(", ""), end.replace("(", "")
            start, end = start.replace(")", ""), end.replace(")", "")
            
#            print(start, end)
        except:
            start = end = words.replace("id", "").replace("(", "").replace(")", "")
#            print(start)
        meeting_dialogues[meeting_name].append((start, end))



#####  Extract complete dialogues and store in files under ../ICSI_plus_NXT/processing/_meeting_id_dialogues.txt file

In [114]:
print(len(meeting_dialogues.keys()))
#print(meeting_dialogues['Bdb001'])

for meeting, sentences in meeting_dialogues.items():
    with open("../data/ICSI_plus_NXT/processing/"+meeting+"_dialogues.txt", "w") as f:
        starts, ends = set(), set()
        for start, end in sentences:
            starts.add(start)
            ends.add(end)
        #go through entire text and start printing if start token is in "starts". stop printing if end token is in "ends"
        print_w = False
        nl = False
        for word_id, word in meeting_dict[meeting].items():
            if word_id in starts:
                print_w = True
            if word_id in ends:
                print_w = False
                nl = True
            if print_w:
                if nl:
                    f.write("\n")
                    nl = False
                f.write(word+ " ")


75


### Extract summaries

In [29]:
for dialog_f in sorted(glob.glob("/home/rajivn/W266/W266-fall-2020-hwu-rnair/data/ICSI_plus_NXT/ICSIplus/DialogueActs/*acts.xml")):
    meeting_name = dialog_f.split("/")[-1].split(".")[0]
    

In [72]:
"Bdb001.A.words.xml#id(Bdb001.w.1,362)..id(Bdb001.disfmarker.56)".split("..")

['Bdb001.A.words.xml#id(Bdb001.w.1,362)', 'id(Bdb001.disfmarker.56)']

In [73]:
"Bdb001.A.words.xml#id(Bdb001.w.1,476)".split("..")

['Bdb001.A.words.xml#id(Bdb001.w.1,476)']